#### Import modules

In [58]:
import sys
import os
from numpy import arange
from pandas import read_csv, DataFrame, Series
from matplotlib.pyplot import savefig
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

utils_path = os.path.abspath(os.path.join('..', 'utils'))
sys.path.append(utils_path)

from dslabs_functions import ts_aggregation_by, series_train_test_split, plot_forecasting_eval, plot_forecasting_series, series_train_test_split
from dslabs_functions import HEIGHT, plot_multiline_chart
from dslabs_functions import FORECAST_MEASURES, DELTA_IMPROVE, plot_multiline_chart

#### Load data

In [59]:
data_path = os.path.abspath(os.path.join('..','..', 'data_copy/forecast_ny_arrests.csv'))
if not os.path.exists(data_path):
    print("File not found")
    data_path = os.path.abspath(os.path.join('..','..', 'data/f_s_forecast_ny_arrests.csv'))

def load_data(data_path):
    data: DataFrame = read_csv(
        data_path,
        index_col="Date",
        sep=";",
        decimal=".",
        parse_dates=True,
        infer_datetime_format=True,
    )
    return data

all_data = load_data(data_path)
file_tag = "ARREST"
target = "Manhattan"



data = all_data[[target]]


File not found


In [60]:
from pandas import DataFrame

def dataframe_train_test_split(data: DataFrame, trn_pct: float = 0.90) -> tuple[DataFrame, DataFrame]:
    trn_size = int(len(data) * trn_pct)
    train = data.iloc[:trn_size]
    test = data.iloc[trn_size:]
    return train, test


#### Preparation

In [61]:
import pandas as pd

def data_preparation(data):
    # Missing value 
    data: DataFrame = data.dropna(how="any", inplace=False) 
    # Train/test split
    train, test = dataframe_train_test_split(data)   
    
    # Scaling
    transf: MinMaxScaler = MinMaxScaler(feature_range=(0, 1), copy=True).fit(train)
    train_scaled = DataFrame(transf.transform(train), index=train.index, columns=train.columns)
    test_scaled = DataFrame(transf.transform(test), index=test.index, columns=test.columns) 

    # Smoothing
    WIN_SIZE = 10
    train_smooth: Series = train_scaled.rolling(window=WIN_SIZE).mean()
    train_smooth = train_smooth.dropna()

    # Differentiation
    train_diff: Series = train_smooth.diff()
    train_diff = train_diff.dropna()
    test_diff: Series = test_scaled.diff()
    test_diff = test_diff.dropna()

    return train_diff, test_diff
    
    
train_target, test_target = data_preparation(data)

train_all, test_all = data_preparation(all_data)


print(train_target.columns, test_target.columns)
print(train_all.columns, test_all.columns)

Index(['Manhattan'], dtype='object') Index(['Manhattan'], dtype='object')
Index(['Bronx', 'Brooklyn', 'Queens', 'StatenIsland', 'Manhattan'], dtype='object') Index(['Bronx', 'Brooklyn', 'Queens', 'StatenIsland', 'Manhattan'], dtype='object')


In [62]:
from torch import no_grad, tensor
from torch.nn import LSTM, Linear, Module, MSELoss
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset


def prepare_dataset_for_lstm(series, seq_length: int = 4):
    setX: list = []
    setY: list = []
    for i in range(len(series) - seq_length):
        past = series[i : i + seq_length]
        future = series[i + 1 : i + seq_length + 1]
        setX.append(past)
        setY.append(future)
    return tensor(setX), tensor(setY)


class DS_LSTM(Module):
    def __init__(self, train, input_size: int = 1, hidden_size: int = 50, num_layers: int = 1, length: int = 4):
        super().__init__()
        self.lstm = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.linear = Linear(hidden_size, 1)
        self.optimizer = Adam(self.parameters())
        self.loss_fn = MSELoss()

        trnX, trnY = prepare_dataset_for_lstm(train, seq_length=length)
        self.loader = DataLoader(TensorDataset(trnX, trnY), shuffle=True, batch_size=len(train) // 10)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

    def fit(self):
        self.train()
        for batchX, batchY in self.loader:
            y_pred = self(batchX)
            loss = self.loss_fn(y_pred, batchY)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        return loss

    def predict(self, X):
        with no_grad():
            y_pred = self(X)
        return y_pred[:, -1, :]

In [63]:
# train_all.reset_index(drop=True, inplace=True)
train_all

,Bronx,Brooklyn,Queens,StatenIsland,Manhattan
Date,,,,,
2006-01-11,0.053620,0.034496,0.020380,0.021698,0.033069
2006-01-12,0.045023,0.014341,0.047554,0.019811,0.037030
2006-01-13,0.019910,0.019767,0.015761,0.011321,0.020594
2006-01-14,-0.013801,-0.025581,-0.019293,-0.015094,-0.018812
2006-01-15,-0.059729,-0.043798,-0.039402,-0.033962,-0.046337
...,...,...,...,...,...
2020-05-21,0.006561,0.009109,0.013043,-0.006604,0.012277
2020-05-22,-0.002941,0.003101,0.006250,-0.015094,-0.006139
2020-05-23,-0.006787,-0.015504,-0.007065,-0.005660,-0.008713


In [64]:
train_all_series = train_all.values.astype("float32")
test_all_series = test_all.values.astype("float32")

In [65]:

model = DS_LSTM(train_all_series, input_size=5, hidden_size=50, num_layers=1)
loss = model.fit()
print(loss)

tensor(0.0005, grad_fn=<MseLossBackward0>)


In [ ]:
from copy import deepcopy

from matplotlib.pyplot import figure, savefig, subplots


def lstm_study(train, test, nr_episodes: int = 1000, measure: str = "R2"):
    sequence_size = [2, 4, 8]
    nr_hidden_units = [25, 50, 100]

    step: int = nr_episodes // 10
    episodes = [1] + list(range(0, nr_episodes + 1, step))[1:]
    flag = measure == "R2" or measure == "MAPE"
    best_model = None
    best_params: dict = {"name": "LSTM", "metric": measure, "params": ()}
    best_performance: float = -100000

    _, axs = subplots(1, len(sequence_size), figsize=(len(sequence_size) * HEIGHT, HEIGHT))

    for i in range(len(sequence_size)):
        length = sequence_size[i]
        tstX, tstY = prepare_dataset_for_lstm(test, seq_length=length)

        values = {}
        for hidden in nr_hidden_units:
            yvalues = []
            model = DS_LSTM(train, hidden_size=hidden)
            for n in range(0, nr_episodes + 1):
                model.fit()
                if n % step == 0:
                    prd_tst = model.predict(tstX)
                    eval: float = FORECAST_MEASURES[measure](test[length:], prd_tst)
                    print(f"seq length={length} hidden_units={hidden} nr_episodes={n}", eval)
                    if eval > best_performance and abs(eval - best_performance) > DELTA_IMPROVE:
                        best_performance: float = eval
                        best_params["params"] = (length, hidden, n)
                        best_model = deepcopy(model)
                    yvalues.append(eval)
            values[hidden] = yvalues
        plot_multiline_chart(
            episodes,
            values,
            ax=axs[i],
            title=f"LSTM seq length={length} ({measure})",
            xlabel="nr episodes",
            ylabel=measure,
            percentage=flag,
        )
    print(
        f"LSTM best results achieved with length={best_params['params'][0]} hidden_units={best_params['params'][1]} and nr_episodes={best_params['params'][2]}) ==> measure={best_performance:.2f}"
    )
    return best_model, best_params

measure: str = "R2"

train = train_target.values.astype("float32")
test = test_target.values.astype("float32")

best_model, best_params = lstm_study(train, test, nr_episodes=3000, measure=measure)

seq length=2 hidden_units=25 nr_episodes=0 0.0071566104888916016
